# PySuperTuxKart Gymnasium 封装

[![PyPI version](https://badge.fury.io/py/pystk2-gymnasium.svg)](https://badge.fury.io/py/pystk2-gymnasium)

阅读 [更新日志](./CHANGELOG.md)

## 安装

PySuperKart2 gymnasium 封装是一个 Python 包，安装非常简单：

```bash
pip install pystk2-gymnasium
```

请注意，首次运行时，SuperTuxKart 的资源文件会下载到缓存目录。

## AgentSpec

每个受控卡丁车的参数由 `pystk2_gymnasium.AgentSpec` 定义：

- `name`：定义玩家的名称（显示在卡丁车上方）
- `rank_start`：定义起始位置（默认为随机）
- `use_ai`：标志（默认为 False），当为 True 时，忽略操作（在调用 `step` 时，将使用 SuperTuxKart 的 AI 代替动作）
- `camera_mode` 可以设置为 `AUTO`（自动摄像头），`ON`（摄像头开启）或 `OFF`（摄像头关闭）。

## 当前限制

- 不提供图形信息（即无像素图）

## 环境

导入 `pystk2_gymnasium` 后，可以使用以下环境：

- `supertuxkart/full-v0` 是主环境，包含完整的观测数据。观测和动作空间都是字典，包含连续或离散变量（具体结构可通过 `env.observation_space` 和 `env.action_space` 查看）。环境可以通过以下选项进行配置：
  - `agent`：指定为 `AgentSpec`
  - `render_mode`：可以是 None 或 `human`
  - `track`：定义要使用的 SuperTuxKart 赛道（None 为随机赛道）。初始化时调用 `initialize.initialize(with_graphics: bool)` 后，可在 `STKRaceEnv.TRACKS` 中查看完整列表。
  - `num_kart`：赛道上的卡丁车数量（默认为 3）
  - `max_paths`：观测状态中考虑的路径（赛道由路径构成）的最大数量
  - `laps`：圈数（默认为 1）
  - `difficulty`：AI 机器人的难度（最低为 0，最高为 2，默认为 2）

一些环境使用了封装（wrapper），具体说明如下：

- `supertuxkart/simple-v0`（封装：`ConstantSizedObservations`）是简化环境，路径、物品和卡丁车观测数量固定（`state_paths` 默认 5、`state_items` 默认 5、`state_karts` 默认 5）

- `supertuxkart/flattened-v0`（封装：`ConstantSizedObservations`, `PolarObservations`, `FlattenerWrapper`）最大化简化的观测和动作空间，只有 `discrete` 和 `continuous` 两个键

- `supertuxkart/flattened_continuous_actions-v0`（封装：`ConstantSizedObservations`, `PolarObservations`, `OnlyContinuousActionsWrapper`, `FlattenerWrapper`）移除离散动作，仅保留连续的转向/加速

- `supertuxkart/flattened_multidiscrete-v0`（封装：`ConstantSizedObservations`, `PolarObservations`, `DiscreteActionsWrapper`, `FlattenerWrapper`）使用多离散动作空间，由 `acceleration_steps` 和 `steer_steps` 控制加速和转向离散值的数量（默认分别为 5）

- `supertuxkart/flattened_discrete-v0`（封装：`ConstantSizedObservations`, `PolarObservations`, `DiscreteActionsWrapper`, `FlattenerWrapper`, `FlattenMultiDiscreteActions`）完全离散化的动作空间

在时间 `t` 时刻的奖励 $r_t$ 计算如下：

$$ r_{t} =  \frac{1}{10}(d_{t} - d_{t-1}) + (1 - \frac{\mathrm{pos}_t}{K}) \times (3 + 7 f_t) - 0.1 + 10 * f_t $$

其中 $d_t$ 是时间 $t$ 时刻的总赛道距离，$\mathrm{pos}_t$ 是 $K$ 个卡丁车中的当前排名，$f_t$ 为 1 时表示卡丁车完成比赛。

## 封装（Wrappers）

封装可以用于修改环境。

### 固定大小的观测

`pystk2_gymnasium.ConstantSizedObservations(env, state_items=5, state_karts=5, state_paths=5)` 保证观测物品、卡丁车和路径的数量固定。默认每类观测数量为 5。

### 极坐标观测

`pystk2_gymnasium.PolarObservations(env)` 将所有 3D 向量的笛卡尔坐标转换为极坐标（水平角、垂直角、距离）。

### 离散动作

`pystk2_gymnasium.DiscreteActionsWrapper(env, acceleration_steps=5, steer_steps=7)` 将加速和转向动作离散化（默认加速和转向分别为 5 和 7 个值）。

### 平展（动作和观测）

此封装将所有连续和离散空间合并。

`pystk2_gymnasium.FlattenerWrapper(env)` 将**动作和观测**平展。基础环境应是观测空间的字典，转换后的环境是由 `discrete` 和 `continuous` 两个条目组成的字典。

### 平展多离散动作

`pystk2_gymnasium.FlattenMultiDiscreteActions(env)` 将多离散动作空间转换为离散空间，每个可能的动作组合对应一个独立的动作。例如，如果初始空间是 $\{0, 1\} \times \{0, 1, 2\}$，则动作空间变为 $\{0, 1, \ldots, 6\}$。

## 多代理环境

`supertuxkart/multi-full-v0` 可以用于控制多个卡丁车。它接受一个包含多个 `AgentSpec` 的 `agents` 参数。观测和动作是每个卡丁车的单独字典，**字符串**键从 `0` 到 `n-1`，其中 `n` 是卡丁车数量。

可以使用 `MonoAgentWrapperAdapter` 来应用不同的 gymnasium 封装。

以下是一个示例：

```python
from pystk_gymnasium import AgentSpec

agents = [
    AgentSpec(use_ai=True, name="Yin Team", camera_mode=CameraMode.ON),
    AgentSpec(use_ai=True, name="Yang Team", camera_mode=CameraMode.ON),
    AgentSpec(use_ai=True, name="Zen Team", camera_mode=CameraMode.ON)
]

wrappers = [
    partial(MonoAgentWrapperAdapter, wrapper_factories={
        "0": lambda env: ConstantSizedObservations(env),
        "1": lambda env: PolarObservations(ConstantSizedObservations(env)),
        "2": lambda env: PolarObservations(ConstantSizedObservations(env))
    }),
]

make_stkenv = partial(
    make_env,
    "supertuxkart/multi-full-v0",
    render_mode="human",
    num_kart=5,
    agents=agents,
    wrappers=wrappers
)
```

## 动作和观测空间

所有的 3D 向量都基于卡丁车的参考坐标系（`z` 为前方，`x` 为左方，`y` 为上方）：

- `distance_down_track`：距离起点的距离
- `energy`：剩余能量
- `front`：卡丁车的前方（3D 向量）
- `attachment`：附着在卡丁车上的物品（奖励箱、香蕉、氮气、大氮气、小氮气、泡泡糖、复活节彩蛋）
- `attachment_time_left`：物品保留的剩余时间
- `items_position`：物品位置（3D 向量）
- `items_type`：物品类型
- `jumping`：卡丁车是否在跳跃
- `karts_position`：其他卡丁车的位置，从前方的卡丁车开始
- `max_steer_angle`：最大转向角（取决于当前速度）
- `center_path_distance`：到赛道中心的距离
- `center_path`：指向赛道中心的向量
- `paths_start`，`paths_end`，`paths_width`：路径的起点和终点（3D 向量）及路径宽度
- `paths_distance`：路径起点和终点的距离（2 维向量）
- `powerup`：收集的增强物
- `shield_time`
- `skeed_factor`
- `velocity`：速度向量

## 示例

```python
import gymnasium as gym
from pystk2_gymnasium import AgentSpec

# STK gymnasium 使用一个进程
if __name__ == '__main__':
  # 使用平展版的观测和动作空间
  env = gym.make

("supertuxkart/flattened-v0", render_mode="human", agent=AgentSpec(use_ai=False))

  ix = 0
  done = False
  state, *_ = env.reset()

  while not done:
      ix += 1
      action = env.action_space.sample()
      state, reward, terminated, truncated, _ = env.step(action)
      done = truncated or terminated

  # 停止 STK 进程
  env.close()
```

### **`action_space` 和 `observation_space`：**

---

### **1. `action_space`**

```python
Dict(
    'acceleration': Box(0.0, 1.0, (1,), float32),
    'brake': Discrete(2),
    'drift': Discrete(2),
    'fire': Discrete(2),
    'nitro': Discrete(2),
    'rescue': Discrete(2),
    'steer': Box(-1.0, 1.0, (1,), float32)
)
```

#### **解释：**
`action_space` 是一个字典（`Dict` 类型），表示代理可以执行的动作。这个字典中，每个键对应一个动作维度或子动作，具体如下：

- **`acceleration`:** 
  - 类型：`Box(0.0, 1.0, (1,), float32)`
  - 取值范围：连续值 `[0.0, 1.0]`
  - 表示油门的强度，`0.0` 为不加速，`1.0` 为最大加速。

- **`brake`:**
  - 类型：`Discrete(2)`
  - 取值范围：离散值 `{0, 1}`
  - 表示是否踩刹车，`0` 表示不刹车，`1` 表示刹车。

- **`drift`:**
  - 类型：`Discrete(2)`
  - 取值范围：离散值 `{0, 1}`
  - 表示是否漂移，`0` 表示不漂移，`1` 表示漂移。

- **`fire`:**
  - 类型：`Discrete(2)`
  - 取值范围：离散值 `{0, 1}`
  - 表示是否使用攻击道具，`0` 表示不攻击，`1` 表示攻击。

- **`nitro`:**
  - 类型：`Discrete(2)`
  - 取值范围：离散值 `{0, 1}`
  - 表示是否使用氮气加速，`0` 表示不使用，`1` 表示使用。

- **`rescue`:**
  - 类型：`Discrete(2)`
  - 取值范围：离散值 `{0, 1}`
  - 表示是否重置位置（例如赛车出轨或卡住时的救援操作）。

- **`steer`:**
  - 类型：`Box(-1.0, 1.0, (1,), float32)`
  - 取值范围：连续值 `[-1.0, 1.0]`
  - 表示方向盘的转向程度，`-1.0` 表示最大向左转，`1.0` 表示最大向右转，`0.0` 表示直行。

---

#### **总结：**
代理的动作由 7 个子动作组成，每个子动作对应赛车游戏中的一种操作。代理需要输出一个字典，包含每个键对应的取值。例如：
```python
action = {
    'acceleration': [0.8],  # 加速 80%
    'brake': 0,            # 不刹车
    'drift': 1,            # 开始漂移
    'fire': 1,             # 使用道具攻击
    'nitro': 0,            # 不使用氮气加速
    'rescue': 0,           # 不触发救援
    'steer': [-0.5]        # 向左转弯 50%
}
```

---

### **2. `observation_space`**

#### **结构：**
```python
Dict(
    'attachment': Discrete(10),
    'attachment_time_left': Box(0.0, inf, (1,), float32),
    'center_path': Box(-inf, inf, (3,), float32),
    'center_path_distance': Box(-inf, inf, (1,), float32),
    'distance_down_track': Box(-inf, inf, (1,), float32),
    'energy': Box(0.0, inf, (1,), float32),
    'front': Box(-inf, inf, (3,), float32),
    'items_position': Sequence(Box(-inf, inf, (3,), float32), stack=False),
    'items_type': Sequence(Discrete(7), stack=False),
    'jumping': Discrete(2),
    'karts_position': Sequence(Box(-inf, inf, (3,), float32), stack=False),
    'max_steer_angle': Box(-1.0, 1.0, (1,), float32),
    'paths_distance': Sequence(Box(0.0, inf, (2,), float32), stack=False),
    'paths_end': Sequence(Box(-inf, inf, (3,), float32), stack=False),
    'paths_start': Sequence(Box(-inf, inf, (3,), float32), stack=False),
    'paths_width': Sequence(Box(0.0, inf, (1,), float32), stack=False),
    'powerup': Discrete(11),
    'shield_time': Box(0.0, inf, (1,), float32),
    'skeed_factor': Box(0.0, inf, (1,), float32),
    'velocity': Box(-inf, inf, (3,), float32)
)
```

#### **解释：**
`observation_space` 是一个字典，包含许多环境信息，主要描述赛车当前的状态以及赛道周围的特征。

---

##### **主要字段解释：**

1. **`attachment`:**
   - 类型：`Discrete(10)`
   - 表示当前装备的道具类型，取值范围 `[0, 9]`。

2. **`attachment_time_left`:**
   - 类型：`Box(0.0, inf, (1,), float32)`
   - 表示当前装备道具的剩余有效时间。

3. **`center_path`:**
   - 类型：`Box(-inf, inf, (3,), float32)`
   - 表示赛道中心线的方向向量（三维向量）。

4. **`center_path_distance`:**
   - 类型：`Box(-inf, inf, (1,), float32)`
   - 表示赛车距离赛道中心线的横向距离。

5. **`distance_down_track`:**
   - 类型：`Box(-inf, inf, (1,), float32)`
   - 表示赛车沿赛道前进方向的累计距离。

6. **`energy`:**
   - 类型：`Box(0.0, inf, (1,), float32)`
   - 表示赛车的当前能量值（例如氮气或加速能量）。

7. **`front`:**
   - 类型：`Box(-inf, inf, (3,), float32)`
   - 表示赛车的前方方向向量。

8. **`items_position`:**
   - 类型：`Sequence(Box(-inf, inf, (3,), float32), stack=False)`
   - 表示赛道上物品的位置信息，每个物品用三维向量表示。

9. **`items_type`:**
   - 类型：`Sequence(Discrete(7), stack=False)`
   - 表示赛道上物品的类型，取值范围 `[0, 6]`。

10. **`jumping`:**
    - 类型：`Discrete(2)`
    - 表示赛车是否处于跳跃状态，`0` 表示未跳跃，`1` 表示跳跃中。

11. **`karts_position`:**
    - 类型：`Sequence(Box(-inf, inf, (3,), float32), stack=False)`
    - 表示其他赛车的位置信息，每辆车用三维向量表示。

12. **`max_steer_angle`:**
    - 类型：`Box(-1.0, 1.0, (1,), float32)`
    - 表示当前赛车方向盘的最大可转动角度范围。

13. **`paths_distance`:**
    - 类型：`Sequence(Box(0.0, inf, (2,), float32), stack=False)`
    - 表示赛道路径上分支的距离信息。

14. **`paths_start` 和 `paths_end`:**
    - 类型：`Sequence(Box(-inf, inf, (3,), float32), stack=False)`
    - 分别表示路径分支的起点和终点位置。

15. **`paths_width`:**
    - 类型：`Sequence(Box(0.0, inf, (1,), float32), stack=False)`
    - 表示赛道路径的宽度信息。

16. **`powerup`:**
    - 类型：`Discrete(11)`
    - 表示当前携带的道具种类，取值范围 `[0, 10]`。

17. **`shield_time`:**
    - 类型：`Box(0.0, inf, (1,), float32)`
    - 表示护盾的剩余时间。

18. **`skeed_factor`:**
    - 类型：`Box(0.0, inf, (1,), float32)`
    - 表示赛车的打滑系数。

19. **`velocity`:**
    - 类型：`Box(-inf, inf, (3,), float32)`
   

 - 表示赛车的速度向量。

---

#### **总结：**

- **`action_space`** 描述了代理的可操作性，包括加速、刹车、漂移、攻击等赛车操作。
- **`observation_space`** 提供了赛车环境的详细信息，包括赛道中心线、道具信息、其他赛车的位置等。